In [0]:
import os
import tweepy as tw
import pandas as pd
import json
import requests

In [0]:
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [0]:
auth = tw.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
api = tw.API(auth, wait_on_rate_limit=True)

In [0]:
# Make a defined function to open the URL
import requests

r = requests.get('https://raw.githubusercontent.com/jeehobdata/hatebase_dw/master/hatebase/terms_2019-11-20.json')

In [0]:
result = json.loads(r.text)

In [7]:
terms_df = pd.DataFrame(result)
terms_df.head()

,vocabulary_id,term,hateful_meaning,nonhateful_meaning,is_unambiguous,is_unambiguous_in,average_offensiveness,language,plural_of,variant_of,transliteration_of,is_about_nationality,is_about_ethnicity,is_about_religion,is_about_gender,is_about_sexual_orientation,is_about_disability,is_about_class,number_of_sightings,number_of_sightings_this_year,number_of_sightings_this_month,created_on,updated_on
0,LbWTcqCRU,чернильница,A Caucasian woman who sleeps with dark-skinned...,inkpot,False,,NaN,rus,None,None,None,False,True,False,True,False,False,False,13,13,0,2019-10-12 20:34:37,2019-11-04 19:04:23
1,QgCzdCQBw,kanaka,[1] An offensive term for Pacific Islander [2]...,Kanaka maoli can refer in a non-pejorative sen...,False,,91.0,eng,None,None,None,False,True,False,False,False,False,False,19,19,0,2018-11-01 19:42:18,2019-11-01 19:14:03
2,zzaVUeKA8,cracker,"A poor Appalachian or poor Southerner, a white...",[1] Snack [2] Hacker [3] American musical grou...,False,,57.0,eng,None,None,None,False,True,False,False,False,False,True,1881,748,3,2013-04-18 17:34:47,2019-10-30 22:57:19
3,niaHtmNyk,ABCs,[1] American-born Chinese [2] Australian-born ...,Letters of the English alphabet,False,,31.0,eng,ABC,None,None,True,True,False,False,False,False,False,0,0,0,2018-11-01 20:33:27,2019-10-30 22:55:46
4,rnAN2dkU6,ABC,[1] American-born Chinese [2] Australian-born ...,Letters of the English alphabet,False,,28.0,eng,None,None,None,True,True,False,False,False,False,False,0,0,0,2013-08-20 18:40:47,2019-10-30 22:55:41


In [8]:
len(result)

3633

In [0]:
#Pullout only the hate term variables
hateterms = terms_df['term']

#Add hash tags
hateterms =  '#' + hateterms.astype(str)

# Change the datatype of hateterms from a series to a list
hateterms_1 = hateterms.values.tolist()

In [10]:
# Use For loop function  
search_words = hateterms_1
date_since = '2019-10-22'

results = [] 
for i in hateterms_1:
    for tweet in tw.Cursor(api.search,
                       q = i,
                       since = date_since).items(10): 
        results.append(tweet)     

KeyboardInterrupt: ignored

In [11]:
len(results)

9088

In [0]:
def process_results(results):
    id_list = [tweet.id for tweet in results]
    data_set = pd.DataFrame(id_list, columns=['id'])
    
    #processing Tweet Data
    
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at "] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in results]
    data_set["source"] = [tweet.source for tweet in results]
    
    #Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in results]
    data_set["screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["name"] = [tweet.author.name for tweet in results]
    data_set["created_at"] = [tweet.author.created_at for tweet in results]
    data_set["user_description"] = [tweet.author.description for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]
    
    return data_set

data_set = process_results(results)

In [13]:
data_set.head()

,id,text,created_at,retweet_count,favorite_count,source,user_id,screen_name,name,created_at,user_description,user_location
0,1197467358551400449,We keep learning tips and techniques for an ef...,2019-11-21 10:50:40,0,0,Zoho Social,74646104,Recruise,Recruise India Consulting,2009-09-16 03:52:51,#Recruise #HRconsulting provides #recruitment ...,Bangalore
1,1197447477420384256,RT @sujithneeth: #Vijay65 - இது விஜய்க்கு மட்ட...,2019-11-21 09:31:40,1,0,Twitter for Android,1048183477789544449,sriramvathsan7,Sriramvathsan,2018-10-05 12:09:50,Studying in PSNACET.\nThalapathy fan.,
2,1197272281451565061,#Repost iolaniadams\n• • • • • •\nWaiāhole Str...,2019-11-20 21:55:30,0,0,Instagram,1047263932698648577,Aohe_Mea,ʻAʻohe Mea,2018-10-02 23:15:53,https://t.co/W2BFdyR3vE | #UBI 50%PerCapitaGDP...,#AWorldThatWorksForEveryone 🌏
3,1197257302165749760,#FreeHawaii \nQuit arresting #kanaka\n💔💔💔💔💔💔💔💔...,2019-11-20 20:55:59,0,0,Twitter for Android,3261789056,stonergrandma,🌺OccupyPeace🕊 #ProtectMaunaKea 🌺,2015-06-30 17:22:42,#ResistingHate 🙌🏼 #cannabis 💜#WeAreMaunaKea ...,Cascadia/Kauai
4,1196961497219395585,Books on #Kanaka Maoli (Native #Hawaiians) who...,2019-11-20 01:20:34,0,0,Twitter Web App,156819077,KawikaGuillermo,Kawika Guillermo [Dr. Chris P],2010-06-18 01:17:24,I wrote the novel STAMPED and the academic boo...,"Vancouver, British Columbia"


In [14]:
data_set.tail()

,id,text,created_at,retweet_count,favorite_count,source,user_id,screen_name,name,created_at,user_description,user_location
9083,1197930549798719488,Cut Life Experience with • @adusalon Chop Sess...,2019-11-22 17:31:14,0,0,Instagram,18351472,adusalon,All Dolled Up Salons,2008-12-24 05:05:51,South Florida's Top Black Hair Salon:! We love...,"ÜT: 26.288387,-80.273787"
9084,1197921968399568898,"Oh well, suddenly this drawing turned out real...",2019-11-22 16:57:08,2,3,Twitter Web App,1040033508775878656,MidinhaGoldUwU,Midas Gold,2018-09-13 00:24:46,Programmer / PC Gamer/ Demisexual / Pretty fur...,"Sao Paulo, Brazil"
9085,1197900764120854533,RT @misty79182580: #oc #Sena #friendsOc #Rinu ...,2019-11-22 15:32:52,36,0,Twitter for Android,713973435655974913,Krbs614955,Kbennett,2016-03-27 06:18:26,"21, gamer, artist, YouTuber? #Sonic #DragonBal...","Massachusetts, USA"
9086,1197899205349691394,RT @misty79182580: #oc #Sena #friendsOc #Rinu ...,2019-11-22 15:26:41,36,0,Twitter for iPhone,535948128,Monsterking97,The Monster king97,2012-03-25 00:52:35,"I draw stuff, profile pic by @kenivart",In Space.
9087,1197894976803262464,RT @misty79182580: #ocs #Sena #Rena #twins #si...,2019-11-22 15:09:52,56,0,Twitter for Android,1092888423755444226,TheHunterKrieza,"Krieza, The Hunter",2019-02-05 20:51:19,"Father: @Culture_SAO\nKrieza, The Hunter=💙\nCa...",


In [15]:
data_set.to_csv(index=False)

Output hidden; open in https://colab.research.google.com to view.

In [16]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
from google.colab import files
data_set.to_csv('tw_data_set.csv') 
files.download('tw_data_set.csv')